# mAP 산정을 위한 Cartucho (https://github.com/Cartucho/mAP) 이용하기.
Cartucho 를 이용하기 위해서는 먼저, Cartucho 를 통해 산정할 mAP 를 구하고자, Data가 필요한데,\
제공해야하는 Data는 다음과 같다.
1) Test dataset image 들의 Ground Truth 가 저장된 txt files.\
2) Test dataset image 들을 inference 한 결과가 저장된 txt files.\
그래서 위 두 가지의 파일을 만들기 위한 과정을 아래에 적는다.

In [19]:
%autosave 300

Autosaving every 300 seconds


In [12]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2"


In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.4.1
[]


# 2) Test dataset image 들을 inference 한 결과가 저장된 txt file을 뽑기 위한 과정

In [2]:
# 추론을 통해 얻은 test data들의 결과를 가지고 txt파일로 저장하기 위해 다음과 같이 수행.


import io
import os
import scipy.misc
import numpy as np
import six
import time

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util

%matplotlib inline

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
#   print(im_width, im_height)
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Path to label map file
PATH_TO_LABELS = os.path.join('/home001/yousung.choi/projects/project_coordination/workspace/training_demo/annotations','label_map.pbtxt')

# Number of classes the object detector can identify
NUM_CLASSES = 1

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# 1: {'id': 1, 'name': 'book'},
print(category_index)

{1: {'id': 1, 'name': 'Cursor'}}


In [3]:
start_time = time.time()
tf.keras.backend.clear_session()
# detect_fn = tf.saved_model.load('/home/yousungchoi/classes/1_Ambient_computing/99_project/2_pretest_Ambient_project/workspace/training_demo/exported-models/14th_trial/saved_model')
# detect_fn = tf.saved_model.load('/home001/yousung.choi/projects/project_coordination/workspace/training_demo/exported-models/1st_trial/saved_model')
detect_fn = tf.saved_model.load('/home001/yousung.choi/projects/project_coordination/workspace/training_demo/exported-models/3rd_trial_210323/saved_model')
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')

Elapsed time: 8.76283860206604s


In [4]:
with open(PATH_TO_LABELS, 'r') as f:
    for line in f:
        print(line, end="")

item {
  id: 1
  name: 'Cursor'
}



In [10]:
import os
import xml.etree.ElementTree as ET
import cv2
from tqdm import tqdm


# file_path = '/home001/yousung.choi/projects/project_coordination/workspace/training_demo/images/test'
file_path = '/home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/Rainbow_Test/4_5_setting_alarm_onoff/'
file_names = os.listdir(file_path)
file_names.sort()

resultDir = "/home001/yousung.choi/projects/project_coordination/tools/Cartucho/mAP/input/detection-results"
if not os.path.isdir(resultDir):
    os.mkdir(resultDir)

elapsed = []
labels = ['Cursor']
for i, file in enumerate(tqdm(file_names)):
    if file.endswith('.jpg'):
        # inference image by image
#         print(file)
        image_path = os.path.join(file_path, file)
        image_np = load_image_into_numpy_array(image_path)
        input_tensor = np.expand_dims(image_np, 0)
        start_time = time.time()      
        detections = detect_fn(input_tensor)
        end_time = time.time()
        
        # file name to save
        file_name = file.split('.')
        new_file = resultDir + '/' + file_name[0] + '.txt'
        
        # get output after inference
        true_idx = detections['detection_scores'][0] > 0.9
        scores = detections['detection_scores'][0][true_idx].numpy()
        boxes = detections['detection_boxes'][0][true_idx].numpy() * 640
        boxes = np.around(boxes).astype(int)
        classes = detections['detection_classes'][0][true_idx].numpy().astype(np.int32)
        
#         print(true_idx)
#         print(scores, boxes, classes)

        f = open(new_file, 'w', encoding='UTF8')
        class_list = set()
        
        for idx, one_class in enumerate(classes):
            data = labels[int(one_class) - 1] + " "
            data += str(scores[idx]) + " " + str(boxes[idx][1]) + " " + str(boxes[idx][0]) + " " + str(boxes[idx][3]) + " " + str(boxes[idx][2]) + " " + "\n"
            f.write(data)
        f.close()
    
print("Done")

100%|██████████| 360/360 [02:40<00:00,  2.24it/s]

Done


# 1) Test dataset image 들의 Ground Truth 가 저장된 txt files을 뽑기 위한 과정

In [22]:
# 결과
import cv2
import numpy as np
import os
import xml.etree.ElementTree as ET

import math

targetDir = "/home001/yousung.choi/projects/project_coordination/tools/2_preprocessing_test/Rainbow_Test/13_Qmemo_typing_and_drawing/"
resizeDir = "/home001/yousung.choi/projects/project_coordination/tools/Cartucho/mAP/input/ground-truth/"

if not os.path.isdir(resizeDir):
    os.mkdir(resizeDir)
    
# targetDir에서 filename 을 리스트로 가져오기
_dir = os.listdir(targetDir)
file_list = []
for file in _dir:
    if '.xml' in file:
        temp = file.split(".")
        file_list.append(temp[0])
#         print(temp[0])

for file in tqdm(file_list):

    org_xml_file = targetDir + file + ".xml"
    edit_txt_file = resizeDir + file + ".txt"

    f = open(edit_txt_file, 'w', encoding='UTF8')

    if os.path.isfile(org_xml_file):
        targetXML = open(org_xml_file, 'rt', encoding='UTF8')
        tree = ET.parse(targetXML)
        root = tree.getroot()

        object_list = set()
        # 좌표수정
        for objects in root.iter("object"):
            object_tag = objects.find("name")
            xmin_tag = objects.find("bndbox").find("xmin")
            ymin_tag = objects.find("bndbox").find("ymin")
            xmax_tag = objects.find("bndbox").find("xmax")
            ymax_tag = objects.find("bndbox").find("ymax")
            if object_tag.text in object_list:
                data = str(object_tag.text) + " " + str(xmin_tag.text) + " " + str(ymin_tag.text) + " " + str(
                    xmax_tag.text) + " " + str(ymax_tag.text) + " " + "difficult\n"
#                 print(data)
            else:
                data = str(object_tag.text) + " " + str(xmin_tag.text) + " " + str(ymin_tag.text) + " " + str(
                    xmax_tag.text) + " " + str(ymax_tag.text) + "\n"
#                 print(data)

            object_list.add(object_tag.text)
            f.write(data)

        f.close()
#         print(file + "[success]")
        # xml 저장

100%|██████████| 270/270 [00:00<00:00, 5196.73it/s]


In [1]:
!ls

1_preprocessing_Data_Acquisition_preprocessing.ipynb  Cartucho
9_Cartucho_making_pre_datasets.ipynb		      preprocessing_test


# gloning Cartucho and erase unecessary files
# and copy result files into folders

In [13]:
%%bash
rm -rf Cartucho
mkdir Cartucho
cd Cartucho
git clone https://github.com/Cartucho/mAP
rm -rf mAP/input/detection-results/*
rm -rf mAP/input/ground-truth/*
rm -rf mAP/input/images-optional/*
cp /home001/yousung.choi/projects/project_coordination/workspace/training_demo/images/Cartucho/detection-results/* mAP/input/detection-results
cp /home001/yousung.choi/projects/project_coordination/workspace/training_demo/images/Cartucho/groundtruth/* mAP/input/ground-truth

Cloning into 'mAP'...


In [11]:
%%bash
cd Cartucho/mAP
python3 main.py




88.48% = Cursor AP 
mAP = 88.48%
Figure(640x480)


#### False Negative 를 위한 분석
## Recall 율 때문에 Precision 이 높아지는데, Recall 에 영향을 줄 수 있는 False Negative (정답인데 정답으로 분류가 명확히 되지 않는 녀석들) 로 분류된 인자들이 어떤 것인지 살펴봐야 함
### 방법은, Detection 

In [27]:
import os

# file_dir = "/home001/yousung.choi/projects/project_coordination/tools/Cartucho/mAP/input/detection-results"
file_dir = "/home001/yousung.choi/projects/project_coordination/tools/Cartucho/mAP/input/detection-results_V30"
files = os.listdir(file_dir)

weird_files = []
for file_name in files:
    with open(file_dir + "/" + file_name, 'r') as f:
        for i, line in enumerate(f.readlines()):
            if i > 0:
                weird_files.append(file_name)
#     break    
weird_files.sort()
print(weird_files)

['images1142_resized_linear_0414.txt', 'images1143_resized_linear_0414.txt', 'images1144_resized_linear_0414.txt', 'images1145_resized_linear_0414.txt', 'images1146_resized_linear_0414.txt', 'images1147_resized_linear_0414.txt', 'images1148_resized_linear_0414.txt', 'images1149_resized_linear_0414.txt', 'images1150_resized_linear_0414.txt', 'images1151_resized_linear_0414.txt', 'images1152_resized_linear_0414.txt', 'images1153_resized_linear_0414.txt', 'images1154_resized_linear_0414.txt', 'images13054_resized_linear_0414.txt', 'images13056_resized_linear_0414.txt', 'images13057_resized_linear_0414.txt', 'images13059_resized_linear_0414.txt', 'images13060_resized_linear_0414.txt', 'images13112_resized_linear_0414.txt', 'images13113_resized_linear_0414.txt', 'images13114_resized_linear_0414.txt', 'images13115_resized_linear_0414.txt', 'images13116_resized_linear_0414.txt', 'images13117_resized_linear_0414.txt', 'images13118_resized_linear_0414.txt', 'images13119_resized_linear_0414.txt'

#### What if invalid files (< .5) are treated as vaild (그러니까, 50% 이하로 분류된 녀석들이 FN 로 분류될 수 있는 것인가 Cartucho 에서.)


In [20]:
import os

file_dir = '/home001/yousung.choi/projects/project_coordination/tools/Cartucho/mAP/input/detection-results_Rainbow/'

files = os.listdir(file_dir)

fp_files = set()
for i, file in enumerate(tqdm(files)):
    with open(file_dir + file, 'r') as f:
        for j, line in enumerate(f.readlines()):
            data = line.split()
            if (len(data) < 3):
                print(file)
#     break        
    


100%|██████████| 1154/1154 [00:00<00:00, 49249.86it/s]


In [19]:
import os

file_dir = '/home001/yousung.choi/projects/project_coordination/tools/Cartucho/mAP/input/detection-results_Rainbow/'

files = os.listdir(file_dir)

fp_files = set()
for i, file in enumerate(tqdm(files)):
    with open(file_dir + file, 'r') as f:
        for j, line in enumerate(f.readlines()):
            data = line.split()
            score = float(data[1])
            if score < 0.7:
                print(file, ': ', score)
#     break        
    


100%|██████████| 1154/1154 [00:00<00:00, 39449.90it/s]

images1658_resized_linear_0414.txt :  0.6893851
images4149_resized_linear_0414.txt :  0.6528983
images1470_resized_linear_0414.txt :  0.68327826
images2420_resized_linear_0414.txt :  0.67757064
images2257_resized_linear_0414.txt :  0.6658753
images3296_resized_linear_0414.txt :  0.68886006
images4426_resized_linear_0414.txt :  0.6117384
images4361_resized_linear_0414.txt :  0.62190527
images1540_resized_linear_0414.txt :  0.60498846
images2387_resized_linear_0414.txt :  0.6558724
images13425_resized_linear_0414.txt :  0.68587685


In [15]:
fp_files = set()
for i in range(5):
    fp_files.add(i)
    
print(fp_files)    

for i in range(5):
    fp_files.add(i)
    
print(fp_files)    

{0, 1, 2, 3, 4}
{0, 1, 2, 3, 4}


In [33]:
import os

file_dir = '/home001/yousung.choi/projects/project_coordination/tools/1_datasets/2_video_to_imgs/Rainbow_Test/13_Qmemo_typing_and_drawing/invalid/'

files = os.listdir(file_dir)

for file in files:
    if not file.endswith('.jpg'):
        files.remove(file)

for i, file in enumerate(tqdm(files)):
    image_path = os.path.join(file_dir, file)
    image_np = load_image_into_numpy_array(image_path)
    input_tensor = np.expand_dims(image_np, 0)
    
    detections = detect_fn(input_tensor)
        
    # get output after inference
    true_idx = detections['detection_scores'][0] > 0.1
    break
#     true_idx = detections['detection_scores'][0] > 0.6
#     scores = detections['detection_scores'][0][true_idx].numpy()
#     boxes = detections['detection_boxes'][0][true_idx].numpy() * 640
#     boxes = np.around(boxes).astype(int)
#     classes = detections['detection_classes'][0][true_idx].numpy().astype(np.int32)
    


  0%|          | 0/547 [00:05<?, ?it/s]

tf.Tensor([0.01664126 0.01636073 0.01454771 0.01287827 0.01130536], shape=(5,), dtype=float32)
